In [2]:
import sys
import pickle
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torch.utils.data import DataLoader

from pathlib import Path

HOME = Path.cwd().parent
sys.path.append(str(HOME))

In [3]:
with open(HOME / 'cache/transfer.pkl', 'rb') as f:
    mat2bat = pickle.load(f)

In [4]:
class Dataset:
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, indx: int):
        return self.feature[indx], self.label[indx]

    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

    def to(self, device):
        self.feature = self.feature.to(device)
        self.label = self.label.to(device)
        return self

    @property
    def device(self):
        return self.label.device


class DatasetWithSupport(Dataset):
    def __init__(self, feature, label, support_feat, support_label, support_size: int = 8):
        self.feature = feature
        self.label = label
        self.support_feature = support_feat
        self.support_label = support_label
        self.support_size = support_size

    def __getitem__(self, i: int):
        feat, label = self.feature[i], self.label[i]
        index = random.sample(range(self.support_label.shape[1]), self.support_size)
        sup_feat, sup_label = self.support_feature[i, index], self.support_label[i, index]
        return feat, label, sup_feat, sup_label

    def to(self, device):
        self.feature = self.feature.to(device)
        self.label = self.label.to(device)
        self.support_feature = self.support_feature.to(device)
        self.support_label = self.support_label.to(device)
        return self

In [5]:
class ZscoreProcessor:
    def __init__(
        self,
        label_mean: float | None = None,
        label_std: float | None = None
    ):
        self.label_mean = label_mean
        self.label_std = label_std

    def fit(self, label):
        label = label.float()
        if len(label) == 1:
            self.label_mean, self.label_std = label.item(), 1.
        else:
            self.label_mean = torch.mean(label).item()
            self.label_std = torch.std(label).item()

    def transform(self, label):
        label = label.float()
        label = (label - self.label_mean) / self.label_std
        return label

    def inverse_transform(self, label):
        label = label * self.label_std + self.label_mean
        return label


class FeatureSmoother:
    def __init__(self, ths: float = 10.):
        self.ths = ths

    def transform(self, feature):
        feature[feature.abs() > self.ths] = 0.
        return feature


class DiffCurveCalculator:
    def __init__(self, diff_cycle: int = 9, ths: float = 10.):
        self.diff_cycle = diff_cycle
        self.smoother = FeatureSmoother(ths)

    def transform(self, feature, support_feature=None):
        cycle_diff = feature - feature[:, :, [self.diff_cycle]]
        cycle_diff = self.smoother.transform(cycle_diff)
        if support_feature is not None:
            cell_diff = feature[:, None] - support_feature[None]
            cell_diff = self.smoother.transform(cell_diff)
            return cycle_diff, cell_diff
        return cycle_diff

In [6]:
class ConvModule(nn.Module):
    def __init__(self, din, dout, kernel_size,
                 act_fn: str = 'relu',
                 dropout: float = 0.1):
        nn.Module.__init__(self)
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(din, dout, kernel_size)
        self.pool1 = nn.AvgPool2d(kernel_size)
        self.conv2 = nn.Conv2d(dout, dout, kernel_size)
        self.pool2 = nn.AvgPool2d(kernel_size)
        self.act_fn = getattr(torch, act_fn)
        self.dropout = nn.Dropout2d(dropout)

    def output_shape(self, H, W):
        # conv1 output
        H, W = H - self.kernel_size[0] + 1, W - self.kernel_size[1] + 1
        # pool1 output
        H = int((H - self.kernel_size[0]) / self.kernel_size[0] + 1)
        W = int((W - self.kernel_size[1]) / self.kernel_size[1] + 1)
        # conv2 output
        H, W = H - self.kernel_size[0] + 1, W - self.kernel_size[1] + 1
        # pool2 output
        H = int((H - self.kernel_size[0]) / self.kernel_size[0] + 1)
        W = int((W - self.kernel_size[1]) / self.kernel_size[1] + 1)

        return H, W

    def forward(self, x):
        x = self.conv1(x)
        x = self.act_fn(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act_fn(x)
        x = self.pool2(x)

        return x


class CNNRULPredictor(nn.Module):
    def __init__(self,
                 in_channels: int,
                 channels: int,
                 input_height: int,
                 input_width: int,
                 kernel_size=3,
                 act_fn: str = 'relu'):
        nn.Module.__init__(self)
        self.channels = channels
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)
        if input_height < kernel_size[0]:
            kernel_size = (input_height, kernel_size[1])
        if input_width < kernel_size[1]:
            kernel_size = (kernel_size[0], input_width)
        self.encoder = ConvModule(in_channels, channels, kernel_size, act_fn)
        H, W = self.encoder.output_shape(input_height, input_width)
        self.proj = nn.Conv2d(channels, channels, (H, W))
        self.fc = nn.Linear(channels, 1)

    def forward(self, feature: torch.Tensor):
        x = self.encoder(feature)
        x = self.proj(x)
        x = x.view(-1, self.channels)
        x = torch.relu(x)
        x = self.fc(x).view(-1)

        return x


class BatLiNetRULPredictor(nn.Module):
    def __init__(
        self,
        in_channels: int,
        channels: int,
        input_height: int,
        input_width: int,
        kernel_size: int = 3,
        alpha: float = 0.5,
        act_fn: str = 'relu',
        use_seperate_bias: bool = False
    ):
        nn.Module.__init__(self)
        self.channels = channels
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)
        if input_height < kernel_size[0]:
            kernel_size = (input_height, kernel_size[1])
        if input_width < kernel_size[1]:
            kernel_size = (kernel_size[0], input_width)

        self.ori_enc = ConvModule(in_channels, channels, kernel_size, act_fn)
        H, W = self.ori_enc.output_shape(input_height, input_width)
        self.ori_proj = nn.Conv2d(channels, channels, (H, W))
        self.sup_enc = ConvModule(in_channels, channels, kernel_size, act_fn)
        self.sup_proj = nn.Conv2d(channels, channels, (H, W))
        self.fc = nn.Linear(channels, 1, bias=False)
        self.w1 = nn.Parameter(torch.tensor(0.))
        self.w2 = nn.Parameter(torch.tensor(0.))
        self.use_seperate_bias = use_seperate_bias
        self.alpha = alpha

    def forward(
        self,
        feature: torch.Tensor,
        diff_feature: torch.Tensor,
    ):
        B, S, C, H, W = diff_feature.size()

        x_ori = torch.relu(self.ori_proj(self.ori_enc(feature)))
        x_sup = torch.relu(self.sup_proj(self.sup_enc(diff_feature.view(-1, C, H, W))))

        y_ori = self.fc(x_ori.view(len(feature), self.channels)).view(-1)
        y_sup = self.fc(x_sup.view(B * S, self.channels)).view(B, S)

        if self.use_seperate_bias:
            y_ori += self.w1
            y_sup += self.w2

        return y_ori, y_sup

def mape(x, y):
    x, y = torch.tensor(x), torch.tensor(y)
    return torch.mean((x - y).abs() / y)

In [7]:
support_feat = torch.stack([x.feature for x in mat2bat['LFP'] if 500 < x.label < 2500]).float()
support_label = torch.stack([x.label for x in mat2bat['LFP'] if 500 < x.label < 2500]).float()
nca_feat = torch.stack([x.feature for x in mat2bat['NCA']]).float()
nca_label = torch.stack([x.label for x in mat2bat['NCA']]).float()

In [8]:
N = 2

differ = DiffCurveCalculator()
zscorer = ZscoreProcessor()
zscorer.fit(nca_label[:N])

model = CNNRULPredictor(6, 32, 100, 1000)

# Prepare the data for training and testing
data_train = Dataset(differ.transform(nca_feat[:N]), zscorer.transform(nca_label[:N]))
data_test = Dataset(differ.transform(nca_feat[N:]), nca_label[N:])

model = model.to(0)
data_train = data_train.to(0)
data_test = data_test.to(0)

# Create data loaders
train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = DataLoader(data_test, batch_size=32, shuffle=False)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    if (epoch + 1) % 10 == 0:
        # Evaluate the model on the test data
        model.eval()
        with torch.no_grad():
            ans, labels = [], []
            for inputs, label in test_loader:
                outputs = model(inputs)
                outputs = zscorer.inverse_transform(outputs)
                ans.extend(outputs.cpu().tolist())
                labels.extend(label.cpu().tolist())
            print('Eval MAPE:', mape(ans, labels), ans, labels)

Epoch [1/100], Loss: 0.4858
Epoch [2/100], Loss: 0.2433
Epoch [3/100], Loss: 0.3466
Epoch [4/100], Loss: 0.3671
Epoch [5/100], Loss: 0.1121
Epoch [6/100], Loss: 0.0569
Epoch [7/100], Loss: 0.1514
Epoch [8/100], Loss: 0.0217
Epoch [9/100], Loss: 0.0265
Epoch [10/100], Loss: 0.0868
Eval MAPE: tensor(0.3188) [374.75518798828125, 384.8607482910156, 334.5364074707031, 374.006103515625, 375.4959411621094, 373.84600830078125, 382.6524658203125, 371.6834716796875, 373.2527160644531, 393.2323913574219, 374.43377685546875, 382.50616455078125, 382.820556640625, 382.714111328125, 381.2771911621094, 385.8343200683594, 382.9236755371094, 380.1279602050781, 363.36669921875, 338.3167724609375] [379.0, 445.0, 227.0, 334.0, 297.0, 181.0, 721.0, 383.0, 311.0, 633.0, 264.0, 407.0, 802.0, 1001.0, 338.0, 321.0, 849.0, 332.0, 230.0, 336.0]
Epoch [11/100], Loss: 0.0520
Epoch [12/100], Loss: 0.0057
Epoch [13/100], Loss: 0.0188
Epoch [14/100], Loss: 0.0473
Epoch [15/100], Loss: 0.0377
Epoch [16/100], Loss: 0.00

In [9]:
N = 8

differ = DiffCurveCalculator()
zscorer = ZscoreProcessor()

model = BatLiNetRULPredictor(6, 32, 100, 1000)

# Prepare the data for training and testing
train_feat, diff_feat = differ.transform(nca_feat[:N], support_feat)
test_feat, diff_feat = differ.transform(nca_feat[N:], support_feat)
zscorer.fit(nca_label[:N])
train_label, norm_sup_label = zscorer.transform(nca_label[:N]), zscorer.transform(support_label)
diff_label = train_label[:, None] - norm_sup_label[None]
data_train = DatasetWithSupport(train_feat, train_label, diff_feat, diff_label)
data_test = DatasetWithSupport(
    test_feat,
    nca_label[N:],
    diff_feat,
    norm_sup_label[None].expand(
        len(nca_label[N:]),
        *norm_sup_label.shape
    ),
    32
)

device = 7

model = model.to(device)
data_train = data_train.to(device)
data_test = data_test.to(6)

# Create data loaders
train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = DataLoader(data_test, batch_size=8, shuffle=False)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    for inputs, labels, sup_inputs, sup_labels in train_loader:
        optimizer.zero_grad()
        y1, y2 = model(inputs, sup_inputs)
        loss = 0.2 * criterion(y1, labels.view(-1)) + 0.8 * criterion(y2.view(-1), sup_labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    if (epoch + 1) % 10 == 0:
        # Evaluate the model on the test data
        model.eval()
        with torch.no_grad():
            ans, labels = [], []
            model = model.to(data_test.device)
            for inputs, label, sup_inputs, sup_labels in test_loader:
                y1, y2 = model(inputs, sup_inputs)
                y2 += sup_labels
                y1 = zscorer.inverse_transform(y1)
                y2 = zscorer.inverse_transform(y2)
                for x, y in zip(y1, y2):
                    if y.max() > 0:
                        x = x * 0.8 + y[y > 0].median() * 0.2
                    ans.append(x.item())
                labels.extend(label.cpu().tolist())
            print('Eval MAPE:', mape(ans, labels), ans, labels)
            model = model.to(data_train.device)

Epoch [1/300], Loss: 91.7519
Epoch [2/300], Loss: 41.5122
Epoch [3/300], Loss: 78.0329
Epoch [4/300], Loss: 46.1604
Epoch [5/300], Loss: 28.3820
Epoch [6/300], Loss: 21.9532
Epoch [7/300], Loss: 45.0709
Epoch [8/300], Loss: 48.6917
Epoch [9/300], Loss: 30.3206
Epoch [10/300], Loss: 35.6327
Eval MAPE: tensor(0.2943) [398.51702880859375, 313.70648193359375, 369.6168212890625, 405.514404296875, 324.12530517578125, 336.20550537109375, 351.47125244140625, 424.849365234375, 359.49737548828125, 290.6129150390625, 307.8486633300781, 379.6387939453125, 299.76409912109375, 390.5116882324219] [721.0, 383.0, 311.0, 633.0, 264.0, 407.0, 802.0, 1001.0, 338.0, 321.0, 849.0, 332.0, 230.0, 336.0]
Epoch [11/300], Loss: 25.4799
Epoch [12/300], Loss: 23.8450
Epoch [13/300], Loss: 27.5197
Epoch [14/300], Loss: 32.6828
Epoch [15/300], Loss: 38.1346
Epoch [16/300], Loss: 21.6018
Epoch [17/300], Loss: 24.6382
Epoch [18/300], Loss: 17.4020
Epoch [19/300], Loss: 16.8500
Epoch [20/300], Loss: 27.3331
Eval MAPE: 